In [50]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
import joblib

In [51]:
# Loading dataset
file_path = "../Dados/wfp_food_prices_moz.csv"
dados = pd.read_csv(file_path)
dados['date'] = pd.to_datetime(dados['year'].astype(str) + '-' + dados['month'].astype(str) + '-01')

In [52]:
print(dados['province'].unique())

['Maputo City' 'Gaza' 'Inhambane' 'Cabo_Delgado' 'Manica' 'Nampula' 'Tete'
 'Sofala' 'Niassa' 'Zambezia' 'Maputo']


In [53]:
# Pre-processing
label_cols = ['province', 'district', 'commodity', 'unit', 'pricetype']
colunas_para_manter = label_cols + ['price(MZN)', 'date']
dados = dados[colunas_para_manter].dropna(subset=['price(MZN)'])

label_encoders = {}
for col in label_cols:
    le = LabelEncoder()
    dados[col] = le.fit_transform(dados[col].astype(str))
    label_encoders[col] = le

In [54]:
# Time Features
dados['month'] = dados['date'].dt.month
dados['year'] = dados['date'].dt.year
dados['month_id'] = (dados['year'] - dados['year'].min()) * 12 + dados['month']

In [55]:
# Treining final model (Random Forest)
X = dados.drop(columns=[col for col in ['price(MZN)', 'date', 'category', 'market'] if col in dados.columns])
y = dados['price(MZN)']
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X, y)

RandomForestRegressor(random_state=42)

In [56]:
# Simulate new input for forecast
nova_entrada = pd.DataFrame({
    'province': [label_encoders['province'].transform(['Maputo City'])[0]],
    'district': [label_encoders['district'].transform(['Cidade_De_Maputo'])[0]],
    'commodity': [label_encoders['commodity'].transform(['Maize (white)'])[0]],
    'unit': [label_encoders['unit'].transform(['KG'])[0]],
    'pricetype': [label_encoders['pricetype'].transform(['Retail'])[0]],
    'month': [1],  # Janeiro
    'year': [2026],
    'month_id': [(2026 - dados['year'].min()) * 12 + 1]
})

previsao = rf_model.predict(nova_entrada)[0]
print(f"Preço previsto para Maize (white) em Maputo City (Jan/2026): {previsao:.2f} MZN")


Preço previsto para Maize (white) em Maputo City (Jan/2026): 32.44 MZN


In [57]:
# Exporting model and label encoders
joblib.dump(rf_model, "../Modelo/modelo_rf.pkl")
joblib.dump(label_encoders, "../Modelo/label_encoders.pkl")


['../Modelo/label_encoders.pkl']